install libraries

In [10]:
# from google.colab import drive
# drive.mount('Oppositional_thinking')

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [2]:
# test gpu:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
torch.cuda.device_count()

cuda


2

In [3]:
import os
import torch
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from torch.optim import Adam, RMSprop
from transformers  import  get_scheduler
from sklearn.model_selection import KFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import KFold
import wandb
from datetime import datetime
import os
import shutil
import random
import numpy as np
from itertools import product

2024-05-29 08:53:40.334674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 08:53:40.334786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 08:53:40.465900: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


utils

In [44]:
# utils.py
def set_seed(seed):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)

    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)

def remove_previous_model(folder):
    dirs = [x for x in os.listdir(folder) if os.path.isdir(folder+os.sep+x)]
    for x in dirs:
        shutil.rmtree(folder+os.sep+x, ignore_errors=False, onerror=None)


def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in product(*vals):
        yield dict(zip(keys, instance))

# mydataset.py
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None, mode='train'):
        self.encodings = encodings
        if labels is None:
            self.labels = None  # No labels present
        else:
            self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])  # Or another key that's always present
    
# datareader.py
import pandas as pd
import json

BINARY_MAPPING_CRITICAL_POS = {'CONSPIRACY': 0, 'CRITICAL': 1}
BINARY_MAPPING_CONSPIRACY_POS = {'CRITICAL': 0, 'CONSPIRACY': 1}

CATEGORY_MAPPING_CRITICAL_POS_INVERSE = {0: 'CONSPIRACY', 1: 'CRITICAL'}
CATEGORY_MAPPING_CONSPIRACY_POS_INVERSE = {0: 'CRITICAL', 1: 'CONSPIRACY'}

# TRAIN_DATASET_ES="/kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_es_train.json"
# TRAIN_DATASET_EN="/kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_en_train.json"
TEST_DATASET_EN ="/kaggle/input/dataset-oppositional/test/dataset_en_official_test_nolabels.json"
TEST_DATASET_ES ="/kaggle/input/dataset-oppositional/test/dataset_es_official_test_nolabels.json"


class PAN24Reader:
    def __init__(self):
        pass
    def read_json_file(self, path):
        dataset=[]
        print(f'Loading official JSON {path} dataset')
        with open(path, 'r', encoding='utf-8') as file:
            dataset = json.load(file)
        return dataset

    def load_dataset_classification(self, path, string_labels=False, positive_class='conspiracy'):
        dataset = self.read_json_file(path)
        # convert to a format suitable for classification
        texts = pd.Series([doc['text'] for doc in dataset])
        if string_labels:
            classes = pd.Series([doc['category'] for doc in dataset])
        else:
            if positive_class == 'conspiracy':
                binmap = BINARY_MAPPING_CONSPIRACY_POS
            elif positive_class == 'critical':
                binmap = BINARY_MAPPING_CRITICAL_POS
            else:
                raise ValueError(f'Unknown positive class: {positive_class}')
#             classes = [binmap[doc['category']] for doc in dataset]
#             classes = pd.Series(classes)
        ids = pd.Series([doc['id'] for doc in dataset])
#         data = pd.DataFrame({"text": texts, "id": ids, "label": classes})
        data = pd.DataFrame({"text": texts, "id": ids})

        return data


myReader=PAN24Reader()
es_test_df = myReader.load_dataset_classification(TEST_DATASET_ES, string_labels=False, positive_class='conspiracy')
en_test_df = myReader.load_dataset_classification(TEST_DATASET_EN, string_labels=False, positive_class='conspiracy')


Loading official JSON /kaggle/input/dataset-oppositional/test/dataset_es_official_test_nolabels.json dataset
Loading official JSON /kaggle/input/dataset-oppositional/test/dataset_en_official_test_nolabels.json dataset


fine_tunning.py

In [45]:
def training(_wandb, _model, _train_data, _val_data, _learning_rate, _optimizer_name, _schedule, _epochs,
             _tokenizer, _batch_size=32, _padding="max_length", _max_length=512, _truncation=True,
             _patience=10, _measure= "accuracy", _out=None):
    train_encodings = _tokenizer(_train_data["text"].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")
    val_encodings = _tokenizer(_val_data["text"].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")

    train, val = MyDataset(train_encodings, _train_data["label"].tolist()), MyDataset(val_encodings, _val_data["label"].tolist())

    train_dataloader, val_dataloader  = torch.utils.data.DataLoader(train, batch_size=_batch_size, shuffle=True), torch.utils.data.DataLoader(val, batch_size=_batch_size)


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    _wandb.log({"divice": str(device)})
    if use_cuda:
        model = _model.to(device)
        if torch.cuda.device_count() > 1:
            print(f"Usando {torch.cuda.device_count()} GPUs")
            model = torch.nn.DataParallel(model)
    best_measure, best_model_name, patience = None, None, 0
    training_stats = []
    
    # train_eval = evaluate.load("accuracy")
    train_eval = evaluate.load(f"Yeshwant123/{_measure}")

    lr_scheduler, optimizer = None, None
    #Here we can specify different methods to optmize the paarameters, initially we can consider Adam and RmsProp

    _wandb.log({"info": "Creating the Optimizer and Schedule "})

    lr_scheduler, optimizer = None, None
    if _optimizer_name == "adam":
        optimizer = Adam(_model.parameters(), lr=_learning_rate)
    elif _optimizer_name == "rmsprop":
        optimizer = RMSprop(_model.parameters(), lr=_learning_rate)

    #Here we can define different learning rate schedules, to variate de learning rate in each training step. Initially we use
    # can use linear learning rate schedule
    num_training_steps = _epochs * len(_train_data)
    if _schedule=="linear":
        lr_scheduler = get_scheduler(_schedule, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


    for epoch in range(_epochs):
        if patience >= _patience: break
        total_loss_train, total_acc_train = 0, 0
        total_train_step = 0
        
        _model.train()
        
        for batch in train_dataloader:
            total_train_step += 1
            # print("Epoch ", epoch, "Batch", i)
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            if loss.dim() > 0:
                loss = loss.mean()  # Ensure the loss is a scalar
            loss.backward()
            total_loss_train += loss.item()
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            train_eval.add_batch(predictions=predictions, references=batch["labels"])
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        total_acc_train = train_eval.compute()

        total_eval_steps = 0
        total_loss_val, total_acc_val = 0, 0
        eval_metric = evaluate.load(f"Yeshwant123/{_measure}")
        model.eval()
        for batch in val_dataloader:
            total_eval_steps += 1
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
                loss = outputs.loss
                if loss.dim() > 0:
                    loss = loss.mean()  # Ensure the loss is a scalar
                total_loss_val += loss.item()
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
            eval_metric.add_batch(predictions=predictions, references=batch["labels"])
        total_acc_val = eval_metric.compute()

        if best_measure is None or (best_measure < total_acc_val[_measure]):  # here you must set your save weights
            if best_measure == None: _wandb.log({"info": "It's the first time (epoch) ******************"})
            elif best_measure < total_acc_val[_measure]:
                _wandb.log({"info": "In this epoch an improvement was achieved. (epoch) ******************"})

            best_measure = total_acc_val[_measure]
            try:
                os.makedirs(_out + os.sep + 'models', exist_ok=True)
            except OSError as error:
                _wandb.log({"info": "Directory '%s' can not be created"})
            # remove the directories
            remove_previous_model(_out + os.sep + 'models')
            best_model_name = _out + os.sep + 'models/bestmodel_epoch_{}'.format(epoch +1)
            _wandb.log({"info": "The current best model is " + best_model_name + " "+ str(best_measure)})

            os.makedirs(best_model_name, exist_ok=True)
            if isinstance(model, torch.nn.DataParallel):
                model.module.save_pretrained(best_model_name)
            else:
                model.save_pretrained(best_model_name)
            patience = 0
        else:
            patience += 1
        training_stats.append(
            {
                'epoch': epoch + 1,
                'Training Loss': total_loss_train / total_train_step,
                'Valid. Loss': total_loss_val / total_eval_steps,
                f'Valid.{_measure}': total_acc_val[_measure],
                f'Training.{_measure}': total_acc_train[_measure]
            }
        )
        
        _wandb.log({
            'epoch': epoch + 1,
            'train_loss': total_loss_train / len(train_dataloader),
            f'train_{_measure}': total_acc_train[_measure],
            'val_loss': total_loss_val / len(val_dataloader),
            f'val_{_measure}': total_acc_val[_measure]
        })

    if best_model_name != None:
        if isinstance(model, torch.nn.DataParallel):
            model = model.module.from_pretrained(best_model_name)
        else:
            model = model.from_pretrained(best_model_name)
        _wandb.log({"info": "The final model used to predict the labels of the testing datasets is " + best_model_name})

    df_stats = pd.DataFrame(data=training_stats)
    df_stats = df_stats.set_index('epoch')
    df_stats.to_csv(_out + os.sep + "training_stats.csv")

    _wandb.log({"info": df_stats})
    myplot = sns.lineplot(data=df_stats, palette="tab10", linewidth=2.5)
    fig = myplot.get_figure()
    fig.savefig(_out + os.sep + 'loss-figue.png')
    plt.close()
    return model

############################################################################################################################################################################3
#VALIDATION ON THE TEST SET

# Define the validation function
def validate(_wandb, _model, _test_data, _tokenizer, _batch_size=32, _padding="max_length", _max_length=512, _truncation=True, _measure="accuracy", evaltype=False):
    test_encodings = _tokenizer(_test_data['text'].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")
    
    # Comprobar si hay etiquetas en los datos, asumir que no si no están presentes
    if 'label' in _test_data:
        labels = _test_data['label'].tolist()
    else:
        labels = None  # No labels provided
    
    test = MyDataset(test_encodings, labels, mode="predict" if labels is None else "train")
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=_batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    _model.to(device)
    if torch.cuda.device_count() > 1:
        print(f"Usando {torch.cuda.device_count()} GPUs")
        _model = torch.nn.DataParallel(_model)

    _model.eval()
    all_predictions = []

    with torch.no_grad():
        for batch in test_dataloader:
            batch = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            outputs = _model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            all_predictions.append(predictions.cpu())  # Move predictions to CPU

    # Concatenar todas las predicciones
    all_predictions = torch.cat(all_predictions).numpy()  # Ensure tensor is on CPU before converting to numpy
    return all_predictions

BAGGING

## ENGLISH

In [47]:
import os
import numpy as np
import wandb
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import matthews_corrcoef
from collections import Counter

# Define SEED for reproducibility
SEED = 1234

# Initialize WandB
wandb.login()

# Initialize matrix to hold predictions for each fold (800 samples x 5 folds)
num_folds = 5
num_samples = len(en_test_df)
fold_predictions = np.zeros((num_samples, num_folds))

lang = "english"
model_name = "roberta-base"
    
# Initialize WandB run context
run_name = f"{lang}_{model_name}"
api = wandb.Api()

for fold in range(num_folds):
    # Load the model artifact from WandB
    artifact = api.artifact(f'davidandreuroqueta/lnr_oppositional_thinking_2024-05-28_17-09-22/english_roberta-base_run_3_fold_{fold}_model:latest')
    artifact_dir = artifact.download()
    model_path = artifact_dir

    # Load the model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.eval()

    predictions = validate(_wandb=wandb, _model=model, _test_data=en_test_df, _tokenizer=tokenizer, _batch_size=64, _padding="max_length", _max_length=128, _truncation=True, _measure="mcc", evaltype=False)

    fold_predictions[:, fold] = predictions

# Compute the majority vote for each sample
final_preds = [Counter(fold_predictions[i, :]).most_common(1)[0][0] for i in range(num_samples)]

wandb: Downloading large artifact english_roberta-base_run_3_fold_0_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:6.1


Usando 2 GPUs


wandb: Downloading large artifact english_roberta-base_run_3_fold_1_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:4.5


Usando 2 GPUs


wandb: Downloading large artifact english_roberta-base_run_3_fold_2_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:4.0


Usando 2 GPUs


wandb: Downloading large artifact english_roberta-base_run_3_fold_3_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:3.7


Usando 2 GPUs


wandb: Downloading large artifact english_roberta-base_run_3_fold_4_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:5.9


Usando 2 GPUs


In [51]:
# Convert numerical predictions to labels
def transform_label(pred):
    return "CONSPIRACY" if pred == 1 else "CRITICAL"

# Create DataFrame with predictions
predictions_df = pd.DataFrame(fold_predictions, columns=[f'fold_{i}' for i in range(num_folds)])
predictions_df = predictions_df.map(transform_label)
predictions_df['majority_vote'] = list(map(transform_label, final_preds))
predictions_df['id'] = en_test_df['id']

# Save DataFrame to CSV
output_csv_path = "en_test_predictions.csv"
predictions_df.to_csv(output_csv_path, index=False)

# Create JSON with the required format
json_output = []
for index, row in predictions_df.iterrows():
    json_output.append({"id": row['id'], "category": row['majority_vote']})

json_output_path = "en_test_predictions.json"
with open(json_output_path, 'w') as json_file:
    json.dump(json_output, json_file, indent=2)

# Display DataFrame to user (for Jupyter Notebook environment)
# import ace_tools as tools; tools.display_dataframe_to_user(name="Predictions DataFrame", dataframe=predictions_df)

print(f"Predictions have been saved to {output_csv_path}")
print(f"JSON output has been saved to {json_output_path}")

Predictions have been saved to predictions.csv
JSON output has been saved to predictions.json


## SPANISH

In [53]:
# Define SEED for reproducibility
SEED = 1234

# Initialize WandB
wandb.login()

# Initialize matrix to hold predictions for each fold (800 samples x 5 folds)
num_folds = 5
num_samples = len(es_test_df)
fold_predictions = np.zeros((num_samples, num_folds))

    
all_predictions = []
api = wandb.Api()

for fold in range(num_folds):
    # Load the model artifact from WandB
    artifact = api.artifact(f'ghernandez-hector/lnr_oppositional_thinking_2024-05-28_12-47-06/spanishrobertaadam-base-bne_run8fold{fold}_model:v0')
    artifact_dir = artifact.download()
    model_path = artifact_dir

    # Load the model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.eval()

    predictions = validate(_wandb=wandb, _model=model, _test_data=es_test_df, _tokenizer=tokenizer, _batch_size=64, _padding="max_length", _max_length=128, _truncation=True, _measure="mcc", evaltype=False)

    all_predictions.append(predictions)
    
final_preds = [Counter(fold_predictions[i, :]).most_common(1)[0][0] for i in range(num_samples)]

wandb: Downloading large artifact spanishrobertaadam-base-bne_run8fold0_model:v0, 478.92MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:4.8


Usando 2 GPUs


wandb: Downloading large artifact spanishrobertaadam-base-bne_run8fold1_model:v0, 478.92MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:4.2


Usando 2 GPUs


wandb: Downloading large artifact spanishrobertaadam-base-bne_run8fold2_model:v0, 478.92MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:3.5


Usando 2 GPUs


wandb: Downloading large artifact spanishrobertaadam-base-bne_run8fold3_model:v0, 478.92MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:6.6


Usando 2 GPUs


wandb: Downloading large artifact spanishrobertaadam-base-bne_run8fold4_model:v0, 478.92MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:3.2


Usando 2 GPUs


In [54]:
# Convert numerical predictions to labels
def transform_label(pred):
    return "CONSPIRACY" if pred == 1 else "CRITICAL"

# Create DataFrame with predictions
predictions_df = pd.DataFrame(fold_predictions, columns=[f'fold_{i}' for i in range(num_folds)])
predictions_df = predictions_df.map(transform_label)
predictions_df['majority_vote'] = list(map(transform_label, final_preds))
predictions_df['id'] = es_test_df['id']

# Save DataFrame to CSV
output_csv_path = "es_test_predictions.csv"
predictions_df.to_csv(output_csv_path, index=False)

# Create JSON with the required format
json_output = []
for index, row in predictions_df.iterrows():
    json_output.append({"id": row['id'], "category": row['majority_vote']})

json_output_path = "es_test_predictions.json"
with open(json_output_path, 'w') as json_file:
    json.dump(json_output, json_file, indent=2)

# Display DataFrame to user (for Jupyter Notebook environment)
# import ace_tools as tools; tools.display_dataframe_to_user(name="Predictions DataFrame", dataframe=predictions_df)

print(f"Predictions have been saved to {output_csv_path}")
print(f"JSON output has been saved to {json_output_path}")

Predictions have been saved to es_test_predictions.csv
JSON output has been saved to es_test_predictions.json
